In [1]:
import pandas as pd
import numpy as np
import seaborn as sn

In [40]:
df_e=pd.read_csv("employee_data.csv")
df_i=pd.read_csv("insurance_data.csv")
df_v=pd.read_csv("vendor_data.csv")

In [3]:
df_e.shape , df_i.shape , df_v.shape

((1200, 10), (10000, 38), (600, 7))

In [4]:
df_e.info() , df_i.info() , df_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   AGENT_ID            1200 non-null   object
 1   AGENT_NAME          1200 non-null   object
 2   DATE_OF_JOINING     1200 non-null   object
 3   ADDRESS_LINE1       1200 non-null   object
 4   ADDRESS_LINE2       179 non-null    object
 5   CITY                1193 non-null   object
 6   STATE               1200 non-null   object
 7   POSTAL_CODE         1200 non-null   int64 
 8   EMP_ROUTING_NUMBER  1200 non-null   int64 
 9   EMP_ACCT_NUMBER     1200 non-null   object
dtypes: int64(2), object(8)
memory usage: 93.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TXN_DATE_TIME             10000 non-null  o

(None, None, None)

In [ ]:
sorted(list(df_e.columns)), # AGENTS

['ADDRESS_LINE1',
 'ADDRESS_LINE2',
 'AGENT_ID',
 'AGENT_NAME',
 'CITY',
 'DATE_OF_JOINING',
 'EMP_ACCT_NUMBER',
 'EMP_ROUTING_NUMBER',
 'POSTAL_CODE',
 'STATE']

In [ ]:
sorted(list(df_i.columns))

['ACCT_NUMBER',
 'ADDRESS_LINE1',
 'ADDRESS_LINE2',
 'AGE',
 'AGENT_ID',
 'ANY_INJURY',
 'AUTHORITY_CONTACTED',
 'CITY',
 'CLAIM_AMOUNT',
 'CLAIM_STATUS',
 'CUSTOMER_EDUCATION_LEVEL',
 'CUSTOMER_ID',
 'CUSTOMER_NAME',
 'EMPLOYMENT_STATUS',
 'HOUSE_TYPE',
 'INCIDENT_CITY',
 'INCIDENT_HOUR_OF_THE_DAY',
 'INCIDENT_SEVERITY',
 'INCIDENT_STATE',
 'INSURANCE_TYPE',
 'LOSS_DT',
 'MARITAL_STATUS',
 'NO_OF_FAMILY_MEMBERS',
 'POLICE_REPORT_AVAILABLE',
 'POLICY_EFF_DT',
 'POLICY_NUMBER',
 'POSTAL_CODE',
 'PREMIUM_AMOUNT',
 'REPORT_DT',
 'RISK_SEGMENTATION',
 'ROUTING_NUMBER',
 'SOCIAL_CLASS',
 'SSN',
 'STATE',
 'TENURE',
 'TRANSACTION_ID',
 'TXN_DATE_TIME',
 'VENDOR_ID']

In [ ]:
sorted(list(df_v.columns))

['ADDRESS_LINE1',
 'ADDRESS_LINE2',
 'CITY',
 'POSTAL_CODE',
 'STATE',
 'VENDOR_ID',
 'VENDOR_NAME']

In [6]:
{ i for i in df_i.columns}.intersection({i for i in df_e.columns}).intersection({i for i in df_v.columns})

{'ADDRESS_LINE1', 'ADDRESS_LINE2', 'CITY', 'POSTAL_CODE', 'STATE'}

In [7]:
# pd.concat([df_i.ADDRESS_LINE1,df_i.ADDRESS_LINE2],axis=1)
x=pd.DataFrame(df_i.ADDRESS_LINE1+df_i.ADDRESS_LINE2)

In [8]:
x.drop_duplicates()

,0
0,7701 West Saint John Road#2010
1,NaN
8,642 South 2nd Street#608
12,12870 Old Seward Highway#STE 000105
21,1358 Scenic AvenueR B
...,...
9398,525 Merrill Lane#308
9462,269 Park Street#1
9464,721 South Jackson Street#103
9666,322 East Taylor Street#1103


In [9]:
x.isnull().value_counts()

True     8505
False    1495
dtype: int64

## #1

In [10]:
ddf=df_i.merge(df_e.merge(df_v,how="left",on='ADDRESS_LINE1'),how="left",on='ADDRESS_LINE1')

In [11]:
ddf.shape

(10640, 53)

In [12]:
df_i.INSURANCE_TYPE.unique()

array(['Health', 'Property', 'Travel', 'Life', 'Motor', 'Mobile'],
      dtype=object)

## #2

In [13]:
df_i.CLAIM_STATUS.unique()

array(['A', 'D'], dtype=object)

In [14]:
df_i.groupby(by=df_i.INSURANCE_TYPE)["CLAIM_AMOUNT"].sum().sort_values(ascending=False)[:3]

INSURANCE_TYPE
Life        91478000
Property    41579000
Health      18254000
Name: CLAIM_AMOUNT, dtype: int64

### due to no info on Claim Status it can be divided to A or D status

## #3

In [15]:
df_i.RISK_SEGMENTATION.unique()

array(['L', 'M', 'H'], dtype=object)

In [16]:
df_i[ df_i.RISK_SEGMENTATION == "H"].groupby(by=df_i.STATE)["CLAIM_AMOUNT"].sum().head(5)

STATE
AK    1251200
AL    1855500
AR    1794700
AZ    1879500
CA    3396500
Name: CLAIM_AMOUNT, dtype: int64

## #4

In [17]:
def yes_no(v):
    if v[0] == v[1] == v[2]:
        return 1
    else:
        return 0

In [18]:
ddf["COLOCATION"]=ddf[["INCIDENT_STATE","STATE","STATE_x"]].apply(yes_no,axis=1)

In [19]:
ddf.COLOCATION.value_counts(normalize=True)

0    0.976504
1    0.023496
Name: COLOCATION, dtype: float64

In [20]:
ddf.COLOCATION.mean()

0.023496240601503758

In [24]:
df_i.POLICE_REPORT_AVAILABLE.value_counts()

1    6928
0    3072
Name: POLICE_REPORT_AVAILABLE, dtype: int64

## #5

In [22]:
ddf.AUTHORITY_CONTACTED=ddf[["AUTHORITY_CONTACTED","POLICE_REPORT_AVAILABLE"]].apply(lambda v : "Police" if v[1]==1 and v[0] != "Police" else v[0])

## #6

In [42]:
df_i.TXN_DATE_TIME=pd.to_datetime(df_i.TXN_DATE_TIME)

In [44]:
last_30_devia=df_i.set_index(df_i.TXN_DATE_TIME).groupby("INSURANCE_TYPE")["CLAIM_AMOUNT"].rolling("30d").mean().reset_index()

In [45]:
last_30_devia.rename(columns={"CLAIM_AMOUNT":"avg_30_CLAIM_AMOUNT"},inplace=True)

In [103]:
df_ii=df_i[["INSURANCE_TYPE","TXN_DATE_TIME","CLAIM_AMOUNT","CUSTOMER_NAME"]].merge(last_30_devia,how='left',on=["INSURANCE_TYPE","TXN_DATE_TIME"])

In [104]:
df_ii.drop_duplicates(subset=["INSURANCE_TYPE","TXN_DATE_TIME","CLAIM_AMOUNT","CUSTOMER_NAME"],keep='first',inplace=True)

In [105]:
x=pd.to_datetime('2020-06-01')

In [106]:
x

Timestamp('2020-06-01 00:00:00')

In [107]:
def claim_dev(v):
    g=v[0]/v[3]
    if ((v[2] - x)/np.timedelta64(1,'D')) < 30.00 :
        return -1
    elif g < 0.5:
        return 1
    else:
        return 0

In [108]:
df_ii["CLAIM_DEVI"] = df_ii[["avg_30_CLAIM_AMOUNT","INSURANCE_TYPE","TXN_DATE_TIME","CLAIM_AMOUNT"]].apply(claim_dev,axis=1)

In [109]:
df_ii

,INSURANCE_TYPE,TXN_DATE_TIME,CLAIM_AMOUNT,CUSTOMER_NAME,avg_30_CLAIM_AMOUNT,CLAIM_DEVI
0,Health,2020-06-01,9000,Christopher Demarest,9000.000000,-1
8,Property,2020-06-01,26000,Ricardo Gatlin,26000.000000,-1
16,Property,2020-06-01,13000,Lashawn Engles,26000.000000,-1
24,Health,2020-06-01,16000,Steven Bassett,9000.000000,-1
32,Travel,2020-06-01,3000,Jason Rodriguez,3000.000000,-1
...,...,...,...,...,...,...
52339,Motor,2021-06-30,1000,Patrick Hauman,5558.333333,0
52344,Life,2021-06-30,67000,Paula Noel,52202.898551,0
52350,Health,2021-06-30,8000,Luis Clark,11007.575758,0
52352,Motor,2021-06-30,2000,Ryan Gordon,5558.333333,0


## #7

In [187]:
tdf=ddf[["CLAIM_AMOUNT","INSURANCE_TYPE","AGENT_NAME","CLAIM_STATUS"]]

In [219]:
vv=tdf[tdf["CLAIM_STATUS"] == 'A'].groupby(by=["AGENT_NAME"]).agg({"INSURANCE_TYPE":pd.Series.nunique ,"CLAIM_AMOUNT":"sum"})

In [224]:
vv[vv.INSURANCE_TYPE > 2].sort_values(by="CLAIM_AMOUNT",ascending=False)

,INSURANCE_TYPE,CLAIM_AMOUNT
AGENT_NAME,,
Amy King,3,252000
James Harrison,4,232000
John Lolley,4,223700
Louise Turney,3,223100
Robin Mrozinski,5,218400
...,...,...
Ken Tantillo,4,6900
Candace Oleson,3,6700
Jeremy Fields,3,5700


In [225]:
# vv.apply(lambda x: x["INSURANCE_TYPE"] > 2 )["CLAIM_AMOUNT"].sum()

## #8

In [256]:
tdf1=ddf[["CUSTOMER_NAME","PREMIUM_AMOUNT","INSURANCE_TYPE"]]

In [261]:
def change(v):
    if v[0] == "Mobile" or v[0] == "Travel":
        return v[1]-((v[1]*10)/100)
    elif v[0] == "Health" or v[0] == "Property":
        return v[1]+((v[1]*7)/100)
    elif v[0] == 'Life' or  v[0] == 'Motor':
        return v[1]+((v[1]*2)/100)

In [290]:
tdf1.loc[:,"PREMIUM_AMOUNT_CHANGED"]=tdf1.loc[:,["INSURANCE_TYPE","PREMIUM_AMOUNT"]].apply(change,axis=1)

/tmp/ipykernel_3628/4231049865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf1.loc[:,"PREMIUM_AMOUNT_CHANGED"]=tdf1.loc[:,["INSURANCE_TYPE","PREMIUM_AMOUNT"]].apply(change,axis=1)


In [291]:
tdf1.shape , tdf1.CUSTOMER_NAME.nunique()

((10640, 4), 9880)

In [266]:
r1=tdf1.groupby("CUSTOMER_NAME").agg({"PREMIUM_AMOUNT_CHANGED":"sum","PREMIUM_AMOUNT":"sum"})

In [268]:
r1["change_in_percent"]=100*((r1.PREMIUM_AMOUNT_CHANGED -r1.PREMIUM_AMOUNT )/abs(r1.PREMIUM_AMOUNT))

In [269]:
r1

,PREMIUM_AMOUNT_CHANGED,PREMIUM_AMOUNT,change_in_percent
CUSTOMER_NAME,,,
Aaron Carter,159.4835,149.05,7.0
Aaron Guay,120.7495,112.85,7.0
Aaron Hearnen,72.7974,71.37,2.0
Aaron Hernandez,136.6176,127.68,7.0
Aaron Klingensmith,62.3160,69.24,-10.0
...,...,...,...
Zachery Edwards,62.2800,69.20,-10.0
Zane Garner,144.4500,135.00,7.0
Zenaida Norman,84.6294,82.97,2.0


## #9

In [299]:
ddf["ELIGIBLE_FOR_DISCOUNT"]=np.where(((ddf.TENURE > 60) & (ddf.EMPLOYMENT_STATUS == 'N') & (ddf.NO_OF_FAMILY_MEMBERS >= 4)), 1 , 0)

In [302]:
ddf.ELIGIBLE_FOR_DISCOUNT.mean()

0.029793233082706767

## #10

In [48]:
last_30_veloc=df_i.set_index(df_i.TXN_DATE_TIME).groupby("INSURANCE_TYPE")["CLAIM_AMOUNT"].rolling("30d").count().reset_index()

In [50]:
last_3_veloc=df_i.set_index(df_i.TXN_DATE_TIME).groupby("INSURANCE_TYPE")["CLAIM_AMOUNT"].rolling("3d").count().reset_index()

In [81]:
last_30_veloc.rename(columns={"CLAIM_AMOUNT" : "LAST_30_VELOC" },inplace=True)
last_3_veloc.rename(columns={"CLAIM_AMOUNT" : "LAST_3_VELOC" },inplace=True)

In [82]:
tdf11=df_i[["INSURANCE_TYPE","TXN_DATE_TIME","CLAIM_AMOUNT","CUSTOMER_NAME"]].merge(last_3_veloc,how="left",left_on=["INSURANCE_TYPE","TXN_DATE_TIME"],right_on=["INSURANCE_TYPE","TXN_DATE_TIME"])

In [87]:
tdf11.drop_duplicates(subset=["INSURANCE_TYPE","TXN_DATE_TIME","CLAIM_AMOUNT","CUSTOMER_NAME"],keep='first',inplace=True)

In [86]:
tdf12=df_i[["INSURANCE_TYPE","TXN_DATE_TIME","CLAIM_AMOUNT","CUSTOMER_NAME"]].merge(last_30_veloc,how="left",left_on=["INSURANCE_TYPE","TXN_DATE_TIME"],right_on=["INSURANCE_TYPE","TXN_DATE_TIME"])

In [88]:
tdf12.drop_duplicates(subset=["INSURANCE_TYPE","TXN_DATE_TIME","CLAIM_AMOUNT","CUSTOMER_NAME"],keep='first',inplace=True)

In [99]:
res=tdf11.merge(tdf12,how="inner")

In [101]:
res["Claim_velocity"]=res.LAST_30_VELOC/res.LAST_3_VELOC

In [102]:
res

,INSURANCE_TYPE,TXN_DATE_TIME,CLAIM_AMOUNT,CUSTOMER_NAME,LAST_3_VELOC,LAST_30_VELOC,Claim_velocity
0,Health,2020-06-01,9000,Christopher Demarest,1.0,1.0,1.000000
1,Property,2020-06-01,26000,Ricardo Gatlin,1.0,1.0,1.000000
2,Property,2020-06-01,13000,Lashawn Engles,1.0,1.0,1.000000
3,Health,2020-06-01,16000,Steven Bassett,1.0,1.0,1.000000
4,Travel,2020-06-01,3000,Jason Rodriguez,1.0,1.0,1.000000
...,...,...,...,...,...,...,...
9995,Motor,2021-06-30,1000,Patrick Hauman,11.0,120.0,10.909091
9996,Life,2021-06-30,67000,Paula Noel,10.0,138.0,13.800000
9997,Health,2021-06-30,8000,Luis Clark,10.0,132.0,13.200000
9998,Motor,2021-06-30,2000,Ryan Gordon,11.0,120.0,10.909091


## #11

In [280]:
ddf.AGENT_NAME.nunique()

1146

In [282]:
ddf.groupby("AGENT_NAME").agg({"CLAIM_AMOUNT":"count"}).sort_values(by="CLAIM_AMOUNT").head((1146*5)//100)

,CLAIM_AMOUNT
AGENT_NAME,
Adam Dixon,1
Denise Newsome,1
Ray Dunlap,1
Dianne Peters,1
Dianne Purcell,1
Don Ritchie,1
Rafael Scott,1
Donna Boucher,1
Donnell Phillips,1


## #12

In [283]:
vvv=ddf.groupby("AGENT_NAME").agg({"CLAIM_AMOUNT":"sum"})

In [287]:
np.where((vvv.CLAIM_AMOUNT >= 15000),1,0).mean()

0.7495636998254799